In [ ]:
# default_exp models.stosa

# STOSA
> Stochastic Self-Attention model for Sequential Item Recommendation.

SR encodes users’ dynamic interests by modeling item-item transition relationships in sequences. Recent advancements in Transformer introduce the self-attention mechanism to reveal the position-wise item-item relationships, which leads to the state-of-the-art performance in SR. SASRec is the pioneering work in proposing Transformer for sequential recommendation, which applies scaled dot-product self-attention to learn item-item correlation weights. BERT4Rec adopts bi-directional modeling in sequences. TiSASRec and SSE-PT extend SASRec with additional time interval information and user regularization, respectively.

**Sto**chastic **S**elf-**A**ttention (STOSA) embeds each item as a stochastic Gaussian distribution, the covariance of which encodes the uncertainty. **Wasserstein Self-Attention module** characterize item-item position-wise relationships in sequences, which effectively incorporates uncertainty into model training. Wasserstein attentions also enlighten the collaborative transitivity learning as it satisfies triangle inequality. **Regularization term** to the ranking loss assures the dissimilarity between positive and the negative items.

<img src='https://github.com/recohut/reco-static/raw/master/media/images/models/stosa.png'>

Model Architecture of the proposed STOSA. $𝑠_𝑖$ denotes the item in the position 𝑖 and $\hat{𝑠}_{𝑖+1}$ indicates the output inferred next item in (𝑖 +1)-th position. We first represent items as stochastic embeddings with Elliptical Gaussian distributions, comprised of the mean embedding and covariance embedding. Then we develop a novel Wasserstein self-attention module based on the Wasserstein distance to infer the stochastic sequence embeddings. A Wasserstein distance is adopted to measure the dissimilarity between items in the sequence with uncertainty signals. Finally, we incorporate a novel regularization term measuring the distance between positive and negative items into the standard BPR loss.

**Why Wasserstein distance?** There are several advantages of using Wasserstein distance. First, Wasserstein distance measures the distance between distributions, with the capability of measuring the dissimilarity of items with uncertainty information. Secondly, Wasserstein distance satisfies triangle inequality and can capture collaborative transitivity inductively in sequence modeling. Finally, Wasserstein distance also enjoys the advantage of a more stable training process as it provides a smoother measurement when two distributions are non-overlapping, which in SR means two items are far away from each other. However, KL divergence will produce an infinity distance, causing numerical instability.

For more information:
1. https://arxiv.org/pdf/2201.06035v1.pdf
2. https://github.com/RecoHut-Stanzas/STOSA

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.nb_imports import *
from fastcore.test import *

In [ ]:
#export
from typing import Any, Iterable, List, Optional, Tuple, Union, Callable

import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.nn import functional as F

from pytorch_lightning import LightningModule

from recohut.models.layers.encoding import DistSAEncoder, DistMeanSAEncoder
from recohut.evaluation.metrics import get_eval_metrics_v2, recall_at_k, ndcg_at_k, cal_mrr
from recohut.utils.distances import wasserstein_distance, kl_distance, wasserstein_distance_matmul

In [ ]:
#export
class STOSA(LightningModule):
    def __init__(self,
                 item_size,
                 num_users,
                 hidden_size,
                 max_seq_length,
                 num_attention_heads,
                 optimizer='adamw',
                 learning_rate = 0.003,
                 hidden_dropout_prob=0.2,
                 attention_probs_dropout_prob=0.2,
                 num_hidden_layers=2,
                 distance_metric='wasserstein',
                 initializer_range=0.02,
                 pvn_weight=0.1,
                 kernel_param=1.0,
                 **kwargs):
        super().__init__()
        self.learning_rate = learning_rate
        self.optimizer = optimizer
        self.item_size = item_size
        self.max_seq_length = max_seq_length
        self.num_users = num_users
        self.hidden_size = hidden_size
        self.hidden_dropout_prob = hidden_dropout_prob
        self.num_attention_heads = num_attention_heads
        self.attention_probs_dropout_prob = attention_probs_dropout_prob
        self.num_hidden_layers = num_hidden_layers
        self.distance_metric = distance_metric
        self.initializer_range = initializer_range
        self.pvn_weight = pvn_weight
        self.kernel_param = kernel_param

        self.item_mean_embeddings = nn.Embedding(item_size, hidden_size, padding_idx=0)
        self.item_cov_embeddings = nn.Embedding(item_size, hidden_size, padding_idx=0)
        self.position_mean_embeddings = nn.Embedding(max_seq_length, hidden_size)
        self.position_cov_embeddings = nn.Embedding(max_seq_length, hidden_size)
        self.user_margins = nn.Embedding(num_users, 1)
        self.item_encoder = DistSAEncoder(hidden_size, num_attention_heads, hidden_dropout_prob, 
                                          attention_probs_dropout_prob, num_hidden_layers, 
                                          distance_metric)
        self.layernorm = nn.LayerNorm(hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(hidden_dropout_prob)

        self.apply(self.init_weights)

    def add_position_mean_embedding(self, sequence):

        seq_length = sequence.size(1)
        position_ids = torch.arange(seq_length, dtype=torch.long, device=sequence.device)
        position_ids = position_ids.unsqueeze(0).expand_as(sequence)
        item_embeddings = self.item_mean_embeddings(sequence)
        position_embeddings = self.position_mean_embeddings(position_ids)
        sequence_emb = item_embeddings + position_embeddings
        sequence_emb = self.layernorm(sequence_emb)
        sequence_emb = self.dropout(sequence_emb)
        elu_act = torch.nn.ELU()
        sequence_emb = elu_act(sequence_emb)

        return sequence_emb

    def add_position_cov_embedding(self, sequence):

        seq_length = sequence.size(1)
        position_ids = torch.arange(seq_length, dtype=torch.long, device=sequence.device)
        position_ids = position_ids.unsqueeze(0).expand_as(sequence)
        item_embeddings = self.item_cov_embeddings(sequence)
        position_embeddings = self.position_cov_embeddings(position_ids)
        sequence_emb = item_embeddings + position_embeddings
        sequence_emb = self.layernorm(sequence_emb)
        elu_act = torch.nn.ELU()
        sequence_emb = elu_act(self.dropout(sequence_emb)) + 1

        return sequence_emb

    def forward(self, input_ids, user_ids):

        attention_mask = (input_ids > 0).long()
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2) # torch.int64
        max_len = attention_mask.size(-1)
        attn_shape = (1, max_len, max_len)
        subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1) # torch.uint8
        subsequent_mask = (subsequent_mask == 0).unsqueeze(1)
        subsequent_mask = subsequent_mask.long()

        extended_attention_mask = extended_attention_mask * subsequent_mask
        extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype) # fp16 compatibility
        extended_attention_mask = (1.0 - extended_attention_mask) * (-2 ** 32 + 1)

        mean_sequence_emb = self.add_position_mean_embedding(input_ids)
        cov_sequence_emb = self.add_position_cov_embedding(input_ids)

        item_encoded_layers = self.item_encoder(mean_sequence_emb,
                                                cov_sequence_emb,
                                                extended_attention_mask,
                                                output_all_encoded_layers=True)

        mean_sequence_output, cov_sequence_output, att_scores = item_encoded_layers[-1]

        margins = self.user_margins(user_ids)
        return mean_sequence_output, cov_sequence_output, att_scores, margins

    def init_weights(self, module):
        """ Initialize the weights.
        """
        if isinstance(module, (nn.Linear, nn.Embedding)):
            # Slightly different from the TF version which uses truncated_normal for initialization
            module.weight.data.normal_(mean=0.01, std=self.initializer_range)
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        if isinstance(module, nn.Linear) and module.bias is not None:
            module.bias.data.zero_()

    def training_step(self, batch, batch_idx):
        user_ids, input_ids, target_pos, target_neg, _ = batch
        sequence_mean_output, sequence_cov_output, att_scores, margins = self(input_ids, user_ids)
        loss, batch_auc, pvn_loss = self.bpr_optimization(sequence_mean_output, sequence_cov_output, target_pos, target_neg)
        #loss, batch_auc, pvn_loss = self.margin_optimization(sequence_mean_output, sequence_cov_output, target_pos, target_neg, margins)
        # loss, batch_auc = self.ce_optimization(sequence_mean_output, sequence_cov_output, target_pos, target_neg)
        self.log("train_loss", loss.item())
        self.log("train_pvn_loss", pvn_loss.item())
        self.log("train_auc", batch_auc.item())
        return loss

    def validation_step(self, batch, batch_idx):
        user_ids, input_ids, target_pos, target_neg, answers = batch
        recommend_mean_output, recommend_cov_output, _, _ = self(input_ids, user_ids)
        loss, batch_auc, pvn_loss = self.bpr_optimization(recommend_mean_output, recommend_cov_output, target_pos, target_neg)
        recommend_mean_output = recommend_mean_output[:, -1, :]
        recommend_cov_output = recommend_cov_output[:, -1, :]
        if self.distance_metric == 'kl':
            rating_pred = self.kl_predict_full(recommend_mean_output, recommend_cov_output)
        else:
            rating_pred = self.dist_predict_full(recommend_mean_output, recommend_cov_output)
        rating_pred = rating_pred.cpu().data.numpy().copy()

        batch_user_index = user_ids.cpu().numpy()
        rating_pred[args.valid_matrix[batch_user_index].toarray() > 0] = 1e+24
        # reference: https://stackoverflow.com/a/23734295, https://stackoverflow.com/a/20104162
        ind = np.argpartition(rating_pred, 40)[:, :40]
        #ind = np.argpartition(rating_pred, -40)[:, -40:]
        arr_ind = rating_pred[np.arange(len(rating_pred))[:, None], ind]
        # ascending order
        arr_ind_argsort = np.argsort(arr_ind)[np.arange(len(rating_pred)), ::]
        #arr_ind_argsort = np.argsort(arr_ind)[np.arange(len(rating_pred)), ::-1]
        batch_pred_list = ind[np.arange(len(rating_pred))[:, None], arr_ind_argsort]

        pred_list = batch_pred_list
        answer_list = answers.cpu().data.numpy()
        val_scores = self.get_full_sort_score(answer_list, pred_list)

        # val_scores = self.get_sample_scores(rating_pred)
        self.log("valid_loss", loss.item())
        self.log("valid_pvn_loss", pvn_loss.item())
        self.log("valid_auc", batch_auc.item())
        self.log("valid_scores", val_scores)

    def test_step(self, batch, batch_idx):
        user_ids, input_ids, target_pos, target_neg, answers = batch
        recommend_mean_output, recommend_cov_output, _, _ = self(input_ids, user_ids)
        loss, batch_auc, pvn_loss = self.bpr_optimization(recommend_mean_output, recommend_cov_output, target_pos, target_neg)
        recommend_mean_output = recommend_mean_output[:, -1, :]
        recommend_cov_output = recommend_cov_output[:, -1, :]
        if self.distance_metric == 'kl':
            rating_pred = self.kl_predict_full(recommend_mean_output, recommend_cov_output)
        else:
            rating_pred = self.dist_predict_full(recommend_mean_output, recommend_cov_output)
        rating_pred = rating_pred.cpu().data.numpy().copy()

        batch_user_index = user_ids.cpu().numpy()
        rating_pred[args.test_matrix[batch_user_index].toarray() > 0] = 1e+24
        # reference: https://stackoverflow.com/a/23734295, https://stackoverflow.com/a/20104162
        ind = np.argpartition(rating_pred, 40)[:, :40]
        #ind = np.argpartition(rating_pred, -40)[:, -40:]
        arr_ind = rating_pred[np.arange(len(rating_pred))[:, None], ind]
        # ascending order
        arr_ind_argsort = np.argsort(arr_ind)[np.arange(len(rating_pred)), ::]
        #arr_ind_argsort = np.argsort(arr_ind)[np.arange(len(rating_pred)), ::-1]
        batch_pred_list = ind[np.arange(len(rating_pred))[:, None], arr_ind_argsort]

        pred_list = batch_pred_list
        answer_list = answers.cpu().data.numpy()
        test_scores = self.get_full_sort_score(answer_list, pred_list)

        # test_scores = self.get_sample_scores(rating_pred)
        self.log("test_loss", loss.item())
        self.log("test_pvn_loss", pvn_loss.item())
        self.log("test_auc", batch_auc.item())
        self.log("test_scores", test_scores)

    def configure_optimizers(self):
        # betas = (self.adam_beta1, self.adam_beta2)
        # optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate, betas=betas, weight_decay=self.weight_decay)
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, patience=10, factor=0.1
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
            "monitor": "train_loss",
        }

    def get_sample_scores(self, pred_list):
        pred_list = pred_list.argsort().argsort()[:, 0]
        HIT_1, NDCG_1, MRR = get_eval_metrics_v2(pred_list, 1)
        HIT_5, NDCG_5, MRR = get_eval_metrics_v2(pred_list, 5)
        HIT_10, NDCG_10, MRR = get_eval_metrics_v2(pred_list, 10)
        HIT_20, NDCG_20, MRR = get_eval_metrics_v2(pred_list, 20)
        HIT_40, NDCG_40, MRR = get_eval_metrics_v2(pred_list, 40)
        post_fix = {
            "HIT@1": HIT_1, "NDCG@1": NDCG_1,
            "HIT@5": HIT_5, "NDCG@5": NDCG_5,
            "HIT@10": HIT_10, "NDCG@10": NDCG_10,
            "HIT@20": HIT_20, "NDCG@20": NDCG_20,
            "HIT@40": HIT_40, "NDCG@40": NDCG_40,
            "MRR": MRR,
        }
        return post_fix

    def get_full_sort_score(self, answers, pred_list):
        recall, ndcg, mrr = [], [], 0
        recall_dict_list = []
        ndcg_dict_list = []
        for k in [1, 5, 10, 15, 20, 40]:
            recall_result, recall_dict_k = recall_at_k(answers, pred_list, k)
            recall.append(recall_result)
            recall_dict_list.append(recall_dict_k)
            ndcg_result, ndcg_dict_k = ndcg_at_k(answers, pred_list, k)
            ndcg.append(ndcg_result)
            ndcg_dict_list.append(ndcg_dict_k)
        mrr, mrr_dict = cal_mrr(answers, pred_list)
        post_fix = {
            "HIT@1": recall[0], "NDCG@1": ndcg[0],
            "HIT@5": recall[1], "NDCG@5": ndcg[1],
            "HIT@10": recall[2], "NDCG@10": ndcg[2],
            "HIT@15": recall[3], "NDCG@15": ndcg[3],
            "HIT@20": recall[4], "NDCG@20": ndcg[4],
            "HIT@40": recall[5], "NDCG@40": ndcg[5],
            "MRR": mrr
       }
        return post_fix

    # def get_pos_items_ranks(self, batch_pred_lists, answers):
    #     num_users = len(batch_pred_lists)
    #     batch_pos_ranks = defaultdict(list)
    #     for i in range(num_users):
    #         pred_list = batch_pred_lists[i]
    #         true_set = set(answers[i])
    #         for ind, pred_item in enumerate(pred_list):
    #             if pred_item in true_set:
    #                 batch_pos_ranks[pred_item].append(ind+1)
    #     return batch_pos_ranks

    # def cross_entropy(self, seq_out, pos_ids, neg_ids):
    #     # [batch seq_len hidden_size]
    #     pos_emb = self.item_embeddings(pos_ids)
    #     neg_emb = self.item_embeddings(neg_ids)
    #     # [batch*seq_len hidden_size]
    #     pos = pos_emb.view(-1, pos_emb.size(2))
    #     neg = neg_emb.view(-1, neg_emb.size(2))
    #     seq_emb = seq_out.view(-1, self.hidden_size) # [batch*seq_len hidden_size]
    #     pos_logits = torch.sum(pos * seq_emb, -1) # [batch*seq_len]
    #     neg_logits = torch.sum(neg * seq_emb, -1)
    #     istarget = (pos_ids > 0).view(pos_ids.size(0) * self.max_seq_length).float() # [batch*seq_len]
    #     loss = torch.sum(
    #         - torch.log(torch.sigmoid(pos_logits) + 1e-24) * istarget -
    #         torch.log(1 - torch.sigmoid(neg_logits) + 1e-24) * istarget
    #     ) / torch.sum(istarget)

    #     auc = torch.sum(
    #         ((torch.sign(pos_logits - neg_logits) + 1) / 2) * istarget
    #     ) / torch.sum(istarget)

    #     return loss, auc

    def predict_sample(self, seq_out, test_neg_sample):
        # [batch 100 hidden_size]
        test_item_emb = self.item_embeddings(test_neg_sample)
        # [batch hidden_size]
        test_logits = torch.bmm(test_item_emb, seq_out.unsqueeze(-1)).squeeze(-1)  # [B 100]
        return test_logits

    def predict_full(self, seq_out):
        # [item_num hidden_size]
        test_item_emb = self.item_embeddings.weight
        # [batch hidden_size ]
        rating_pred = torch.matmul(seq_out, test_item_emb.transpose(0, 1))
        return rating_pred

    def bpr_optimization(self, seq_mean_out, seq_cov_out, pos_ids, neg_ids):
        # [batch seq_len hidden_size]
        activation = nn.ELU()
        pos_mean_emb = self.item_mean_embeddings(pos_ids)
        pos_cov_emb = activation(self.item_cov_embeddings(pos_ids)) + 1
        neg_mean_emb = self.item_mean_embeddings(neg_ids)
        neg_cov_emb = activation(self.item_cov_embeddings(neg_ids)) + 1

        # [batch*seq_len hidden_size]
        pos_mean = pos_mean_emb.view(-1, pos_mean_emb.size(2))
        pos_cov = pos_cov_emb.view(-1, pos_cov_emb.size(2))
        neg_mean = neg_mean_emb.view(-1, neg_mean_emb.size(2))
        neg_cov = neg_cov_emb.view(-1, neg_cov_emb.size(2))
        seq_mean_emb = seq_mean_out.view(-1, self.hidden_size) # [batch*seq_len hidden_size]
        seq_cov_emb = seq_cov_out.view(-1, self.hidden_size) # [batch*seq_len hidden_size]

        if self.distance_metric == 'wasserstein':
            pos_logits = wasserstein_distance(seq_mean_emb, seq_cov_emb, pos_mean, pos_cov)
            neg_logits = wasserstein_distance(seq_mean_emb, seq_cov_emb, neg_mean, neg_cov)
            pos_vs_neg = wasserstein_distance(pos_mean, pos_cov, neg_mean, neg_cov)
        else:
            pos_logits = kl_distance(seq_mean_emb, seq_cov_emb, pos_mean, pos_cov)
            neg_logits = kl_distance(seq_mean_emb, seq_cov_emb, neg_mean, neg_cov)
            pos_vs_neg = kl_distance(pos_mean, pos_cov, neg_mean, neg_cov)

        istarget = (pos_ids > 0).view(pos_ids.size(0) * self.max_seq_length).float() # [batch*seq_len]
        loss = torch.sum(-torch.log(torch.sigmoid(neg_logits - pos_logits + 1e-24)) * istarget) / torch.sum(istarget)

        pvn_loss = self.pvn_weight * torch.sum(torch.clamp(pos_logits - pos_vs_neg, 0) * istarget) / torch.sum(istarget)
        auc = torch.sum(
            ((torch.sign(neg_logits - pos_logits) + 1) / 2) * istarget
        ) / torch.sum(istarget)

        return loss, auc, pvn_loss

    def ce_optimization(self, seq_mean_out, seq_cov_out, pos_ids, neg_ids):
        # [batch seq_len hidden_size]
        activation = nn.ELU()
        pos_mean_emb = self.item_mean_embeddings(pos_ids)
        pos_cov_emb = activation(self.item_cov_embeddings(pos_ids)) + 1
        neg_mean_emb = self.item_mean_embeddings(neg_ids)
        neg_cov_emb = activation(self.item_cov_embeddings(neg_ids)) + 1

        # [batch*seq_len hidden_size]
        pos_mean = pos_mean_emb.view(-1, pos_mean_emb.size(2))
        pos_cov = pos_cov_emb.view(-1, pos_cov_emb.size(2))
        neg_mean = neg_mean_emb.view(-1, neg_mean_emb.size(2))
        neg_cov = neg_cov_emb.view(-1, neg_cov_emb.size(2))
        seq_mean_emb = seq_mean_out.view(-1, self.hidden_size) # [batch*seq_len hidden_size]
        seq_cov_emb = seq_cov_out.view(-1, self.hidden_size) # [batch*seq_len hidden_size]


        #pos_logits = d2s_gaussiannormal(wasserstein_distance(seq_mean_emb, seq_cov_emb, pos_mean, pos_cov), self.kernel_param)
        pos_logits = -wasserstein_distance(seq_mean_emb, seq_cov_emb, pos_mean, pos_cov)
        #neg_logits = d2s_gaussiannormal(wasserstein_distance(seq_mean_emb, seq_cov_emb, neg_mean, neg_cov), self.kernel_param)
        neg_logits = -wasserstein_distance(seq_mean_emb, seq_cov_emb, neg_mean, neg_cov)

        istarget = (pos_ids > 0).view(pos_ids.size(0) * self.max_seq_length).float() # [batch*seq_len]

        loss = torch.sum(
            - torch.log(torch.sigmoid(neg_logits) + 1e-24) * istarget -
            torch.log(1 - torch.sigmoid(pos_logits) + 1e-24) * istarget
        ) / torch.sum(istarget)

        auc = torch.sum(
            ((torch.sign(neg_logits - pos_logits) + 1) / 2) * istarget
        ) / torch.sum(istarget)

        return loss, auc

    def margin_optimization(self, seq_mean_out, seq_cov_out, pos_ids, neg_ids, margins):
        # [batch seq_len hidden_size]
        activation = nn.ELU()
        pos_mean_emb = self.item_mean_embeddings(pos_ids)
        pos_cov_emb = activation(self.item_cov_embeddings(pos_ids)) + 1
        neg_mean_emb = self.item_mean_embeddings(neg_ids)
        neg_cov_emb = activation(self.item_cov_embeddings(neg_ids)) + 1

        # [batch*seq_len hidden_size]
        pos_mean = pos_mean_emb.view(-1, pos_mean_emb.size(2))
        pos_cov = pos_cov_emb.view(-1, pos_cov_emb.size(2))
        neg_mean = neg_mean_emb.view(-1, neg_mean_emb.size(2))
        neg_cov = neg_cov_emb.view(-1, neg_cov_emb.size(2))
        seq_mean_emb = seq_mean_out.view(-1, self.hidden_size) # [batch*seq_len hidden_size]
        seq_cov_emb = seq_cov_out.view(-1, self.hidden_size) # [batch*seq_len hidden_size]

        if self.distance_metric == 'wasserstein':
            pos_logits = wasserstein_distance(seq_mean_emb, seq_cov_emb, pos_mean, pos_cov)
            neg_logits = wasserstein_distance(seq_mean_emb, seq_cov_emb, neg_mean, neg_cov)
            pos_vs_neg = wasserstein_distance(pos_mean, pos_cov, neg_mean, neg_cov)
        else:
            pos_logits = kl_distance(seq_mean_emb, seq_cov_emb, pos_mean, pos_cov)
            neg_logits = kl_distance(seq_mean_emb, seq_cov_emb, neg_mean, neg_cov)
            pos_vs_neg = kl_distance(pos_mean, pos_cov, neg_mean, neg_cov)

        istarget = (pos_ids > 0).view(pos_ids.size(0) * self.max_seq_length).float() # [batch*seq_len]
        loss = torch.sum(torch.clamp(pos_logits - neg_logits, min=0) * istarget) / torch.sum(istarget)
        pvn_loss = self.pvn_weight * torch.sum(torch.clamp(pos_logits - pos_vs_neg, 0) * istarget) / torch.sum(istarget)
        auc = torch.sum(
            ((torch.sign(neg_logits - pos_logits) + 1) / 2) * istarget
        ) / torch.sum(istarget)

        return loss, auc, pvn_loss
    
    def dist_predict_full(self, seq_mean_out, seq_cov_out):
        elu_activation = torch.nn.ELU()
        test_item_mean_emb = self.item_mean_embeddings.weight
        test_item_cov_emb = elu_activation(self.item_cov_embeddings.weight) + 1
        #num_items, emb_size = test_item_cov_emb.shape

        #seq_mean_out = seq_mean_out.unsqueeze(1).expand(-1, num_items, -1).reshape(-1, emb_size)
        #seq_cov_out = seq_cov_out.unsqueeze(1).expand(-1, num_items, -1).reshape(-1, emb_size)

        #if args.distance_metric == 'wasserstein':
        #    return wasserstein_distance(seq_mean_out, seq_cov_out, test_item_mean_emb, test_item_cov_emb)
        #else:
        #    return kl_distance(seq_mean_out, seq_cov_out, test_item_mean_emb, test_item_cov_emb)
        #return d2s_1overx(wasserstein_distance_matmul(seq_mean_out, seq_cov_out, test_item_mean_emb, test_item_cov_emb))
        return wasserstein_distance_matmul(seq_mean_out, seq_cov_out, test_item_mean_emb, test_item_cov_emb)
        #return d2s_gaussiannormal(wasserstein_distance_matmul(seq_mean_out, seq_cov_out, test_item_mean_emb, test_item_cov_emb))

    def kl_predict_full(self, seq_mean_out, seq_cov_out):
        elu_activation = torch.nn.ELU()
        test_item_mean_emb = self.item_mean_embeddings.weight
        test_item_cov_emb = elu_activation(self.item_cov_embeddings.weight) + 1

        num_items = test_item_mean_emb.shape[0]
        eval_batch_size = seq_mean_out.shape[0]
        moded_num_items = eval_batch_size - num_items % eval_batch_size
        fake_mean_emb = torch.zeros(moded_num_items, test_item_mean_emb.shape[1], dtype=torch.float32).to(self.device)
        fake_cov_emb = torch.ones(moded_num_items, test_item_mean_emb.shape[1], dtype=torch.float32).to(self.device)

        concated_mean_emb = torch.cat((test_item_mean_emb, fake_mean_emb), 0)
        concated_cov_emb = torch.cat((test_item_cov_emb, fake_cov_emb), 0)

        assert concated_mean_emb.shape[0] == test_item_mean_emb.shape[0] + moded_num_items

        num_batches = int(num_items / eval_batch_size)
        if moded_num_items > 0:
            num_batches += 1

        results = torch.zeros(seq_mean_out.shape[0], concated_mean_emb.shape[0], dtype=torch.float32)
        start_i = 0
        for i_batch in range(num_batches):
            end_i = start_i + eval_batch_size

            results[:, start_i:end_i] = kl_distance_matmul(seq_mean_out, seq_cov_out, concated_mean_emb[start_i:end_i, :], concated_cov_emb[start_i:end_i, :])
            #results[:, start_i:end_i] = d2s_gaussiannormal(kl_distance_matmul(seq_mean_out, seq_cov_out, concated_mean_emb[start_i:end_i, :], concated_cov_emb[start_i:end_i, :]))
            start_i += eval_batch_size

        #print(results[:, :5])
        return results[:, :num_items]

In [ ]:
#export
class STOSAMean(STOSA):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.item_encoder = DistMeanSAEncoder(self.hidden_size,
                                              self.num_attention_heads,
                                              self.hidden_dropout_prob,
                                              self.attention_probs_dropout_prob,
                                              self.num_hidden_layers)

Tests

In [ ]:
item_size = 10
num_users = 10
hidden_size = 10
max_seq_length = 20
hidden_dropout_prob = 0.2
initializer_range = 0.02
num_hidden_layers = 2
attention_probs_dropout_prob = 0.2
num_attention_heads = 2
distance_metric='wasserstein'
initializer_range = 0.02
optimizer = 'adamw'
learning_rate = 0.003
pvn_weight=0.1
kernel_param=1.0

model = STOSA(item_size, num_users, hidden_size, max_seq_length, num_attention_heads,
              optimizer, learning_rate,
              hidden_dropout_prob, attention_probs_dropout_prob, num_hidden_layers,
              distance_metric, initializer_range, pvn_weight, kernel_param)

input_ids = torch.randint(0, 5, (item_size, hidden_size))
user_ids = torch.randint(0, 5, (item_size, hidden_size))

output = model.forward(input_ids, user_ids)
output_shapes = [list(x.shape) for x in [j for sub in output for j in sub]]

test_eq(output_shapes[0], [10, 10])
test_eq(output_shapes[21], [2, 10, 10])
test_eq(output_shapes[31], [10, 1])

model = STOSAMean(item_size, num_users, hidden_size, max_seq_length, num_attention_heads,
                  optimizer, learning_rate,
                  hidden_dropout_prob, attention_probs_dropout_prob, num_hidden_layers,
                  distance_metric, initializer_range, pvn_weight, kernel_param)

input_ids = torch.randint(0, 5, (item_size, hidden_size))
user_ids = torch.randint(0, 5, (item_size, hidden_size))

output = model.forward(input_ids, user_ids)
output_shapes = [list(x.shape) for x in [j for sub in output for j in sub]]

test_eq(output_shapes[0], [10, 10])
test_eq(output_shapes[21], [2, 10, 10])
test_eq(output_shapes[31], [10, 1])

Example

In [ ]:
class Args:
    def __init__(self):
        self.data_dir = '/content/data'
        self.min_seq_length = 10
        self.max_seq_length = 200
        self.sample_frac = 0.2
        self.num_workers = 2

        self.seed = 42
        self.learning_rate = 1e-4
        self.log_dir = '/content/recommender_logs'
        self.model_dir = '/content/recommender_models'
        self.batch_size = 256
        self.max_epochs = 100
        self.val_epoch = 10
        self.gpus = None
        self.monitor = 'valid_loss'
        self.mode = 'min'

        self.item_size = None
        self.num_users = None
        self.hidden_size = 64
        self.hidden_dropout_prob = 0.5
        self.initializer_range = 0.02
        self.num_hidden_layers = 2
        self.attention_probs_dropout_prob = 0.5
        self.num_attention_heads = 2
        self.distance_metric='wasserstein'
        self.optimizer = 'adamw'
        self.pvn_weight=0.1
        self.kernel_param=1.0

args = Args()

In [ ]:
from recohut.datasets.amazon_beauty import AmazonBeautyDataModule

dm = AmazonBeautyDataModule(**args.__dict__)
dm.prepare_data()

Processing...
Done!


In [ ]:
args.item_size = dm.data.item_size
args.num_users = dm.data.num_users
args.valid_matrix = dm.data.valid_rating_matrix
args.test_matrix = dm.data.test_rating_matrix

In [ ]:
model = STOSA(**args.__dict__)

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping


logger = TensorBoardLogger(save_dir=args.log_dir)

checkpoint_callback = ModelCheckpoint(monitor='valid_loss', mode='min', dirpath=args.model_dir, filename='STOSA')
earlystop_callback = EarlyStopping(monitor="valid_loss", patience=5, mode="min")

trainer = Trainer(max_epochs = args.max_epochs,
                  check_val_every_n_epoch = args.val_epoch,
                  logger = logger, 
                  callbacks = [checkpoint_callback, earlystop_callback],
                  gpus = None)

trainer.fit(model, datamodule=dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name                     | Type          | Params
-----------------------------------------------------------
0 | item_mean_embeddings     | Embedding     | 356 K 
1 | item_cov_embeddings      | Embedding     | 356 K 
2 | position_mean_embeddings | Embedding     | 12.8 K
3 | position_cov_embeddings  | Embedding     | 12.8 K
4 | user_margins             | Embedding     | 484   
5 | item_encoder             | DistSAEncoder | 199 K 
6 | layernorm                | LayerNorm     | 128   
7 | dropout                  | Dropout       | 0     
-----------------------------------------------------------
939 K     Trainable params
0         Non-trainable params
939 K     Total params
3.756     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
trainer.test(model, datamodule=dm)

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_auc': 0.525363028049469,
 'test_loss': 0.6991119384765625,
 'test_pvn_loss': 4.98726224899292,
 'test_scores': {'HIT@1': tensor(0.),
                 'HIT@10': tensor(0.),
                 'HIT@15': tensor(0.),
                 'HIT@20': tensor(0.),
                 'HIT@40': tensor(0.0041),
                 'HIT@5': tensor(0.),
                 'MRR': tensor(0.0001, dtype=torch.float64),
                 'NDCG@1': tensor(0.0041),
                 'NDCG@10': tensor(0.0041),
                 'NDCG@15': tensor(0.0041),
                 'NDCG@20': tensor(0.0041),
                 'NDCG@40': tensor(0.0041),
                 'NDCG@5': tensor(0.0041)}}
--------------------------------------------------------------------------------


[{'test_auc': 0.525363028049469,
  'test_loss': 0.6991119384765625,
  'test_pvn_loss': 4.98726224899292,
  'test_scores': {'HIT@1': tensor(0.),
   'HIT@10': tensor(0.),
   'HIT@15': tensor(0.),
   'HIT@20': tensor(0.),
   'HIT@40': tensor(0.0041),
   'HIT@5': tensor(0.),
   'MRR': tensor(0.0001, dtype=torch.float64),
   'NDCG@1': tensor(0.0041),
   'NDCG@10': tensor(0.0041),
   'NDCG@15': tensor(0.0041),
   'NDCG@20': tensor(0.0041),
   'NDCG@40': tensor(0.0041),
   'NDCG@5': tensor(0.0041)}}]

In [ ]:
model = STOSAMean(**args.__dict__)

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping


logger = TensorBoardLogger(save_dir=args.log_dir)

checkpoint_callback = ModelCheckpoint(monitor='valid_loss', mode='min', dirpath=args.model_dir, filename='STOSA')
earlystop_callback = EarlyStopping(monitor="valid_loss", patience=5, mode="min")

trainer = Trainer(max_epochs = 10,
                  check_val_every_n_epoch = 5,
                  logger = logger, 
                  callbacks = [checkpoint_callback, earlystop_callback],
                  gpus = None)

trainer.fit(model, datamodule=dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Missing logger folder: /content/recommender_logs/default

  | Name                     | Type              | Params
---------------------------------------------------------------
0 | item_mean_embeddings     | Embedding         | 356 K 
1 | item_cov_embeddings      | Embedding         | 356 K 
2 | position_mean_embeddings | Embedding         | 12.8 K
3 | position_cov_embeddings  | Embedding         | 12.8 K
4 | user_margins             | Embedding         | 484   
5 | item_encoder             | DistMeanSAEncoder | 199 K 
6 | layernorm                | LayerNorm         | 128   
7 | dropout                  | Dropout           | 0     
---------------------------------------------------------------
939 K     Trainable params
0         Non-trainable params
939 K     Total params
3.756     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
trainer.test(model, datamodule=dm)

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_auc': 0.5465450286865234,
 'test_loss': 0.6889303922653198,
 'test_pvn_loss': 4.940025806427002,
 'test_scores': {'HIT@1': tensor(0.),
                 'HIT@10': tensor(0.),
                 'HIT@20': tensor(0.),
                 'HIT@40': tensor(0.),
                 'HIT@5': tensor(0.),
                 'MRR': tensor(0.),
                 'NDCG@1': tensor(0.),
                 'NDCG@10': tensor(0.),
                 'NDCG@20': tensor(0.),
                 'NDCG@40': tensor(0.),
                 'NDCG@5': tensor(0.)}}
--------------------------------------------------------------------------------


[{'test_auc': 0.5465450286865234,
  'test_loss': 0.6889303922653198,
  'test_pvn_loss': 4.940025806427002,
  'test_scores': {'HIT@1': tensor(0.),
   'HIT@10': tensor(0.),
   'HIT@20': tensor(0.),
   'HIT@40': tensor(0.),
   'HIT@5': tensor(0.),
   'MRR': tensor(0.),
   'NDCG@1': tensor(0.),
   'NDCG@10': tensor(0.),
   'NDCG@20': tensor(0.),
   'NDCG@40': tensor(0.),
   'NDCG@5': tensor(0.)}}]

In [ ]:
#hide
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d -p recohut

Author: Sparsh A.

Last updated: 2022-01-23 05:56:19

recohut: 0.0.11

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.144+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

torch  : 1.10.0+cu111
IPython: 5.5.0
pandas : 1.1.5
numpy  : 1.19.5

